In [1]:
from gerrychain import (Partition, Graph, MarkovChain,
                        updaters, constraints, accept, metagraph)
from gerrychain.proposals import recom
from gerrychain.constraints import contiguous
from functools import partial

# Generate Dual Graph

In [2]:
graph = Graph.from_json("../data/gerrymandria.json")

# Initial Solution: A Partition class

In [28]:
" Produce an initial solution"

my_updaters = {
    "population": updaters.Tally("TOTPOP"),
    "cut_edges": updaters.cut_edges
}

initial_partition = Partition(
    graph,
    assignment="district",
    updaters=my_updaters
)

# Recomb Markov chain

You do not need this. I put here to just see the solution of the best existing algorithm. (generate spanning tree, remerge, resplit)

In [4]:
" Make the Proposal "

# This should be 8 since each district has 1 person in it.
# Note that the key "population" corresponds to the population updater
# that we defined above and not with the population column in the json file.
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(
    recom,
    pop_col="TOTPOP",
    pop_target=ideal_population,
    epsilon=0.01,
    node_repeats=2
)

In [5]:
" Set up the chain "
recom_chain = MarkovChain(
    proposal=proposal,
    constraints=[contiguous],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=40
)

In [6]:
" Run the chain "

assignment_list = []

for i, item in enumerate(recom_chain):
    print(f"Finished step {i+1}/{len(recom_chain)}", end="\r")
    assignment_list.append(item.assignment)

In [ ]:
" collect the assignment at each step of the chain so that we can watch the chain work in a fun animation "

%matplotlib inline
import matplotlib_inline.backend_inline
import matplotlib.cm as mcm
import matplotlib.pyplot as plt
import networkx as nx
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

frames = []

for i in range(len(assignment_list)):
    fig, ax = plt.subplots(figsize=(8,8))
    pos = {node :(data['x'],data['y']) for node, data in graph.nodes(data=True)}
    node_colors = [mcm.tab20(int(assignment_list[i][node]) % 20) for node in graph.nodes()]
    node_labels = {node: str(assignment_list[i][node]) for node in graph.nodes()}

    nx.draw_networkx_nodes(graph, pos, node_color=node_colors)
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_labels(graph, pos, labels=node_labels)
    plt.axis('off')

    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    image = Image.open(buffer)
    frames.append(image)
    plt.close(fig)

def show_frame(idx):
    clear_output(wait=True)
    display(frames[idx])

slider = widgets.IntSlider(value=0, min=0, max=len(frames)-1, step=1, description='Frame:')
slider.layout.width = '500px'
widgets.interactive(show_frame, idx=slider)

Note

Partition class documentation: https://gerrychain.readthedocs.io/en/latest/full_ref/#partition

graph has 64 nodes 

initial partition has 8 districts

# All You Need: Follow the steps given below

In [ ]:
" 1. Determine Valid Single Flips "

# Generate all valid flips for a given partition subject to the contiguous constraint. 
# An iterator that yields dictionaries representing valid flips.
# contiguous(initial_partition) # checks if subgraphs of districts are connected.

metagraph.all_valid_flips(initial_partition, constraints=[contiguous])

In [ ]:
" 2. Convert the iterator of dictionaries to a list of needed arrays "

In [ ]:
" 3. Compute the reward "

# Computes the deviation of populations from exact equality among parts of the partition. 
# returns dictionary from parts to their deviation
# By “deviation” we mean (actual_value - ideal)/ideal (not the absolute value). ideal is the mean of populations of districts.

constraints.validity.deviation_from_ideal(partition=initial_partition)

In [ ]:
" 4. Convert the output sequence of RL to a dictionary of flips "

# flips: dictionary assigning nodes their new districts
# Let a =  [0, 0, -1, 0, ..., 0, 1] be an example for a list of single flip. Consider output is a set A of single flips.

A = {...} 
flips = {} 
l = 10  # arbitrarily taken as the length of a list. 

for a in A:
    while i < l:
        if a[i] == 1:

.
.
.

In [ ]:
" 5. Define the next state returning the new Partition class from the flips that is performed on the current partition. "

# flips: dictionary assigning nodes their new districts

Partition.initial_partition.flip(flips: Dict)   

In [ ]:
" An alternative way for defining the Next state: update some parts of the assignment and get the next partition. Which one is cheaper? "
#Does not check that every node is still assigned to a part. new_parts (Dict) – dictionary mapping (some) parts to their new sets

initial_partition.update_parts(new_parts: Dict) 

In [ ]:
" Plotting function to compare initial and final solutions "



# Some Other functions that may help later

In [ ]:
# metagraph.all_cut_edge_flips(initial_partition) 
# Generate all possible flips of cut edges in a partition without any constraints. 
# An iterator that yields dictionaries representing the flipped edges.

In [ ]:
# Generates all valid Partitions that differ from the given partition by one flip.
# metagraph.all_valid_states_one_flip_away(partition: Partition, constraints: Union[Iterable[Callable], Callable])

In [ ]:
# returns the set of IDs of all parts that gained or lost a node when compared to the parent partition.
# partition is the proposed next Partition class
# onstraints.contiguity.affected_parts(partition: Partition)

In [ ]:
# metagraph.metagraph_degree(initial_partition, constraints: Union[Iterable[Callable], Callable]) 
# Calculate the degree of the node in the metagraph of the given partition. 
# How many possible valid states are reachable from the state given by partition in a single flip subject to the prescribed constraints..

In [ ]:
# initial_partition.assignment.to_dict()  # The assignment as a {node: part} dictionary.

In [ ]:
#updaters.initialize_exterior_boundaries_as_a_set(initial_partition) # A dictionary mapping each part of a partition to the set of nodes in that part that are on the boundary.
#updaters.boundary_nodes(initial_partition, 'boundary_nodes') # takes a partition class, returns set of boundary nodes